### 아래 파일들은 제거!(했음)
[arrhythmia]

- train set
    - 6으로 시작하는 파일: 1267, 1970, 3131, 3198, 3469, 3618, 4827, 4971, 4979, 5055
    
    - 8로 시작하는 파일: 1879, 2164, 5580, 총 13개 제거(직접 터미널에서 다 한 상태)
    
- validation set
    - 여긴 다 8로 시작하는 파일: 7226, 7281, 8783(근데 여긴 허수가 좀 더 있을수도! 앞에 두개는 아예 생성이 안되고(생성이 안되는 경우, 확실히 2개는 맞음), 8783의 경우는 리드가 부족한 파일), 3개 제거
    
    이 외에도 약 37개(전부 data/train/arrhythmia에서만) 12개의 리드 중 몇 개의 리드의 길이가 짧은 데이터들은 무시하고 학습. 
    
    
[normal]의 경우, 오류 안나는 것 같음.


In [18]:
import os
from glob import glob
import numpy as np
import nibabel as nib
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.autograd import Variable
from torchvision import transforms
import torch.nn as nn
from sklearn.metrics import roc_auc_score, average_precision_score
import tqdm
from operator import add

In [19]:
train_path_arr = './data/train/arrhythmia/'
train_path_nor = './data/train/normal/'
val_path_arr = './data/validation/arrhythmia/'
val_path_nor = './data/validation/normal/'


In [20]:
train_arr_list = os.listdir(train_path_arr)
train_nor_list = os.listdir(train_path_nor)
val_arr_list = os.listdir(val_path_arr)
val_nor_list = os.listdir(val_path_nor)
train_arr_list.sort()
train_nor_list.sort()
val_arr_list.sort()
val_nor_list.sort()

In [21]:
len(train_arr_list), len(train_nor_list), len(val_arr_list), len(val_nor_list)


(17836, 21037, 2229, 2630)

In [17]:
len(train_arr_list) +  len(val_arr_list), len(train_nor_list)+len(val_nor_list)

(20065, 23667)

In [6]:
from ECGXMLReader import ECGXMLReader

def decodefile(filepath, namelist):
    file = []
    for name in namelist:
        file.append(ECGXMLReader(filepath + name, name, augmentLeads = True))
    return file


train_arr = decodefile(train_path_arr, train_arr_list)
train_nor = decodefile(train_path_nor, train_nor_list)#나중에 train끼리 합체
val_arr = decodefile(val_path_arr, val_arr_list)
val_nor = decodefile(val_path_nor, val_nor_list)#val 끼리 합체


In [268]:
name_list = [train_arr, train_nor, val_arr, val_nor]

In [262]:
def find_weirdos(decoded_data):#inputs: train_arr, train_nor, val_arr, val_nor/길이가 이상한 리스트 뽑기
    diff_list = []
    for i, file in enumerate(decoded_data):
        l = []
        arr = sorted(file.getAllVoltages().items())
        for line in arr:
            l.append(line[1])
        l = np.asarray(l)
        if l.shape != (12,5000) and l.shape != (12, 4999):#크기가 (12, 5000), (12, 4999)가 아닌 데이터들의 인덱스 리스트를 추출
            diff_list.append(i)
    print("length: ", len(diff_list))
    return diff_list    

In [264]:
diff_set=[]
for x in namelist:
    diff_set.append(find_weirdos(x))
print(diff_set)
#이 결과로 미루어 보아, train dataset에서만 이상한 길이들이 존재함!

/home/chaeeun/.conda/envs/sce/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


length:  37
length:  0
length:  0
length:  0
[[3390, 3893, 4418, 4465, 4487, 4608, 4746, 5134, 5314, 5345, 5452, 5819, 5840, 5873, 6368, 6425, 6471, 6884, 6943, 7280, 7466, 7524, 7643, 7816, 8128, 8242, 8458, 9204, 9262, 9328, 9462, 9556, 9596, 9761, 10499, 10786, 13109], [], [], []]


In [280]:
def making_arr(data_list, diff_list):#train_arr, train_nor, val_arr, val_nor넣고 돌리기.4096길이로 자르기.
    train_array = []
    for i, file in enumerate(data_list):
        if i in diff_list:# 앞에서 추출한 크기가 상이한 데이터들을 제외하고 array를 생성했습니다. 
            continue
        l = []
        arr = sorted(file.getAllVoltages().items())
        for line in arr:
            l.append(line[1])#나중에 여기서 변수를 받아서 선택적으로 리드를 뽑게 수정가능
        l = np.asarray(l, dtype = object).astype('float32')
        l = l[:, :4096]#나중에 얘도 변수화 하면 편하겠다, .T도 고려!transpose = True이런 식으로 argument
        train_array.append(l)
    return train_array#list로 return
#향후 계획: list끼리 + --> 최종 trainset data 만들기. 향후 파이토치 모델에 넣을 시 shuffle의 작업이 꼭 필요함. 

### making array, save as a file .npy

In [281]:
train_arr_arraylist = making_arr(name_list[0], diff_set[0])
train_nor_arraylist = making_arr(name_list[1], diff_set[1])
val_arr_arraylist = making_arr(name_list[2], diff_set[2])
val_nor_arraylist = making_arr(name_list[3], diff_set[3])

train_array = np.asarray(train_arr_arraylist + train_nor_arraylist)
val_array = np.asarray(val_arr_arraylist + val_nor_arraylist)
s
print(train_array.shape)
print(val_array.shape)

np.save('./data_np/train', train_array, allow_pickle=True, fix_imports=True)
np.save('./data_np/validation', val_array, allow_pickle=True, fix_imports=True)

(38836, 12, 4096)
(4859, 12, 4096)


### label 0, 1

label의 경우, json파일에서 추출후 변형하는 방법도 있겠지만, 주신 데이터가 아예 정상, 비정상이 따로 왔기 때문에 그냥 0, 1로 데이터 개수에 맞춰 리스트를 만든 후, 합쳐서 배열을 만들었습니다. 

In [284]:
train_arr_label = [1]*(17836-37)
train_nor_label = [0]*21037
val_arr_label = [1] * 2229
val_nor_label = [0] * 2630
train_label = np.asarray(train_arr_label + train_nor_label).astype('float32').reshape(-1,1)
val_label = np.asarray(val_arr_label + val_nor_label).astype('float32').reshape(-1,1)
np.save('./label_np/train', train_label, allow_pickle = True, fix_imports = True)
np.save('./label_np/validation', val_label, allow_pickle = True, fix_imports = True)

In [286]:
train_label.shape, val_label.shape

((38836, 1), (4859, 1))

In [283]:
train_label.reshape(-1,1).shape

(38836, 1)